# **Evaluation for the fine-tuned Model (Line chart)**


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import os
import json
from PIL import Image
from datasets import Dataset, load_dataset
import torch
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

In [ ]:
from transformers import AutoProcessor, Pix2StructForConditionalGeneration
model = Pix2StructForConditionalGeneration.from_pretrained("google/deplot")
processor = AutoProcessor.from_pretrained("google/deplot")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

In [ ]:
model.load_state_dict(torch.load(f="../Model_Weights/DePlot_Barcharts(Labels).pth"))

In [ ]:
import json
# Open the JSON file
with open('../Test_Data_Barcharts(Labels)/Finance_BarCharts.json', 'r') as json_file:
    data = json.load(json_file)
text_replacements = ["TITLE", "x-axis: Category, y-axis: Values   ","Value=", "<0x0A> ", "|"]

In [ ]:
text_replacements = ["TITLE", "x-axis: Category, y-axis: Values   ","Value=", "<0x0A> ", "|"]

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    tokens = [token for token in tokens if token.isalnum()]  # Remove non-alphanumeric tokens
    tokens = [token for token in tokens if token not in stop_words]  # Remove stopwords
    return set(tokens)

def get_jaccard_similarity_score(predicted_data,actual_data):
    # stop_words = set(stopwords.words('english'))
    predicted_tokens = preprocess_text(predicted_data)
    actual_tokens = preprocess_text(actual_data)
    intersection = len(predicted_tokens.intersection(actual_tokens))
    union = len(predicted_tokens.union(actual_tokens))
    jaccard_similarity = intersection / union
    return jaccard_similarity

def get_resutlant_dict(input_string, plot_type):
    if plot_type == 'bar':
        try:
            # print("input string", input_string)
            parts = input_string.split('|')
            # print("parts\n\n", parts)
            parts[2]=parts[2].replace("<0x0A> ","")
            result_string = "|".join(parts[2:])

            result_string = result_string.replace("<0x0A> ",",")
            result_string = result_string.replace("|", ":")

            pairs = result_string.split(',')

            # Initialize an empty dictionary
            result_dict = {}
            # print("pairs\n", pairs)
            # Iterate through the pairs and extract key-value information
            for pair in pairs:
                # Split each pair by ':' to separate the key and value
                key, value = pair.split(':')

                # Strip whitespace from key and value
                key = key.strip()
                value = value.strip()

                # Add key-value pair to the dictionary
                # print("key",value)
                result_dict[key] = int(value)  # Assuming values are integers

            # Print the resulting dictionary
            # print(result_dict)
            return result_dict
        except:
            parts = input_string.split('|')
            # print("parts\n\n", parts)
            # parts[2:]
            # parts[2]=parts[2].replace("<0x0A> ","")
            result_string = "".join(parts[2:])
            pattern = r'(\d)(?:\s*<0x0A>\s*)'
            result_string = re.sub(pattern, r',', result_string)
            result_string = result_string.replace("<0x0A> ","")
            # # result_string = result_string.replace("|", ":")
            # # result_string = result_string.replace(",", "")
            # print(f"resultant string:- {result_string}")
            words = result_string.split(',')
            # print(words)
            # Initialize an empty dictionary
            output_dict = {}

            # Iterate through the words to create the dictionary
            for word in words:
                numbers = re.findall(r'\d+', word)
                output_string = re.sub(numbers[0], '', word)
                key = output_string.strip()
                output_dict[key] = int(numbers[0])

            # Print the resulting dictionary
            return output_dict

            
    

In [ ]:
#text_replacements = ["TITLE", "x-axis: Category, y-axis: Values   ","Value=", "<0x0A> ", "|"]
def get_rnss_score(predicted_data,actual_data):
    threshold = 10  # You can adjust this threshold based on your criteria
    predicted_data_dict = get_resutlant_dict(predicted_data, "bar")
    actual_data_dict = get_resutlant_dict(actual_data, "bar")
    feature_evaluation_dict = {}
    for key in actual_data_dict.keys():
        if key in predicted_data_dict:
            actual_value = actual_data_dict[key]
            predicted_value = predicted_data_dict[key]
            # Calculate the absolute difference between the numbers
            absolute_difference = abs(actual_value - predicted_value)
            actual_diff_percentage = (actual_value - predicted_value)/100
            feature_evaluation_dict[key] = actual_diff_percentage
            if absolute_difference <= threshold:
                # print(f"The predicted_value {actual_value} and predicted_value {predicted_value} are close.")
                predicted_data_dict[key] = actual_value
                
                # Convert the dictionary to a formatted string
                predicted_data = ', '.join([f'{key}: {value}' for key, value in predicted_data_dict.items()])
                actual_data = ', '.join([f'{key}: {value}' for key, value in actual_data_dict.items()])
                # Print the resulting string
                # print(predicted_data)        
        
    # for replacement in text_replacements:
    #         predicted_data = predicted_data.replace(replacement, "")
    # for replacement in text_replacements:
    #         actual_data = actual_data.replace(replacement, "")
    print(feature_evaluation_dict)
    jeccard_similarity_score = get_jaccard_similarity_score(predicted_data, actual_data)
    # Calculate RNSS score
    # print(jeccard_similarity_score)
    rnss_score = 1 - jeccard_similarity_score
    return rnss_score, jeccard_similarity_score, feature_evaluation_dict

In [ ]:
def get_text_of_test_image(image_name):
    with open('<you can mention your own test dataset', 'r') as json_file:
        data = json.load(json_file)
    for i in data:
        if i['image'] == image_name:
            return i['text']

In [ ]:
image_name = "Your test image name"
image = Image.open(f"<your test image dir>/{image_name}")
inputs = processor(images=image, text="", return_tensors="pt").to(device)
predictions = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(predictions[0], skip_special_tokens=True))

predicted_data = processor.decode(predictions[0], skip_special_tokens=True)

get_resutlant_dict(predicted_data,"bar")

In [ ]:
# Define the directory where your test images are stored
test_images_dir = "../Test_Data_Barcharts(Labels)"

# Initialize an empty list to store the results
results = []


# Initialize a counter for image numbering
image_number = 1

# Loop through each image in the test folder
for image_name in os.listdir(test_images_dir):
    if image_name.endswith(".png"):
        image_path = os.path.join(test_images_dir, image_name)
        
        # Open and process the image
        image = Image.open(image_path)
        inputs = processor(images=image, text="", return_tensors="pt").to(device)
        predictions = model.generate(**inputs, max_new_tokens=512)
        predicted_data = processor.decode(predictions[0], skip_special_tokens=True)
        
        # Get the actual data for the image
        actual_data = get_text_of_test_image(image_name)
        
        # Calculate the RNSS score
        rnss_score, jeccard_similarity_score, feature_evaluation_dict = get_rnss_score(predicted_data, actual_data)
        
        # Append the result to the list
        result = {
            "image": f"{image_name}",
            "rnss_score": "{:.2f}".format(rnss_score),
            "jeccard_similarity_score": "{:.2f}".format(jeccard_similarity_score),
            "feature_evaluation_dict": feature_evaluation_dict
        }
        results.append(result)
        
        # Increment the image number
        image_number += 1

# Save the results in a JSON file
output_file = "predicted_rnss_scores.json"
with open(output_file, 'w') as json_file:
    json.dump(results, json_file, indent=4)

print(f"RNSS scores saved to {output_file}")


In [ ]:
import pandas as pd
with open("predicted_rnss_scores.json","r") as json_file:
    predicted_data = json.load(json_file)
    
predicted_df = pd.DataFrame(predicted_data)

In [ ]:
predicted_df.to_csv("predicted_evaluation_result.csv")

### Evaluation of the same data with Base Model

In [ ]:
import re
model = Pix2StructForConditionalGeneration.from_pretrained("google/deplot")
processor = AutoProcessor.from_pretrained("google/deplot")

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


def process_image(input_image):
    # Load the uploaded image
    image = Image.open(input_image)
    inputs = processor(images=image, text="Chart Data", return_tensors="pt").to(device)
    predictions = model.generate(**inputs, max_new_tokens=512)
    return processor.decode(predictions[0], skip_special_tokens=True)



# Define the directory where your test images are stored
test_images_dir = "../Test_Data_Barcharts(Labels)"

# Initialize an empty list to store the results
results = []


# Initialize a counter for image numbering
image_number = 1

# Loop through each image in the test folder
for image_name in os.listdir(test_images_dir):
    if image_name.endswith(".png"):
        image_path = os.path.join(test_images_dir, image_name)
        print(image_path)
        # Open and process the image
        # image = Image.open(image_path)
        # inputs = processor(images=image, text="", return_tensors="pt").to(device)
        # predictions = model.generate(**inputs, max_new_tokens=512)
        # predicted_data = processor.decode(predictions[0], skip_special_tokens=True)
        predicted_data = process_image(image_path)
        
        # Get the actual data for the image
        actual_data = get_text_of_test_image(image_name)
        # Calculate the RNSS score
        rnss_score, jeccard_similarity_score,feature_evaluation_dict = get_rnss_score(predicted_data, actual_data)
        
        # Append the result to the list
        result = {
            "image": f"chart_{image_number}.png",
            "rnss_score": "{:.2f}".format(rnss_score),
            "jeccard_similarity_score": "{:.2f}".format(jeccard_similarity_score),
            "feature_evaluation_dict": feature_evaluation_dict
        }
        results.append(result)
        
        # Increment the image number
        image_number += 1

# Save the results in a JSON file
output_file = "actual_rnss_scores.json"
with open(output_file, 'w') as json_file:
    json.dump(results, json_file, indent=4)

print(f"RNSS scores saved to {output_file}")


In [ ]:
import pandas as pd
with open("actual_rnss_scores.json","r") as json_file:
    actual_data = json.load(json_file)
    
actual_df = pd.DataFrame(actual_data)
actual_df

In [ ]:
actual_df.to_csv('actual_rnss_score.csv')